<a href="https://colab.research.google.com/github/uol-mediaprocessing/group-projects-traffic-signs/blob/master/main_traffic-signs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Assisting children while biking

### Gruppe 7: Maximilian Hörnis, Florian Schwarm, Dennis Rupprecht

## Motivation 
- Children are active participants in road traffic
- Traffic signs might not be obvious to them
- Handheld technology device (smartphones etc.) usage is prevalent among children 

## Idea

Recognize and classify traffic signs automatically and display them in
an easy to understand assistive technology for children, without distracting them.

## Tasks
    
- Recognizing and classifiying traffic signs
- Interpretation of images in a way that is
    - a) sufficiently comprehensible for children and
    - b) not distracting.

## Possible Extensions

- Detection of intersections without relying on traffic signs
- Detection of traffic lights and their current position (red, green, yellow)
- Applying the traffic sign classifier to live video (or recordings)
- Integrating the system into a mobile application

## Possible Issues

- Recognizing dirty or occluded signs
- Determining which street a sign refers to and how long it is valid
- Reliable detection of intersections